In [5]:
import schedule
import time
import requests
import pandas as pd
from datetime import datetime, timedelta, date
import pyodbc
from sqlalchemy import create_engine
import mysql.connector

In [6]:
def inserting_sql():
    db_config = {
    'host': 'Localhost',
    'user': 'root',
    'password': '8november2002',
    'database': 'bhavDB',
    }
    connection = mysql.connector.connect(**db_config)
    cursor = connection.cursor()
    engine = create_engine(f"mysql+mysqlconnector://{db_config['user']}:{db_config['password']}@{db_config['host']}/{db_config['database']}")

    current_date=datetime.now()
    date = current_date.strftime('%d%m%Y')
    # print(date)
    dates=[]
    dates.append(current_date.strftime("%d%m%Y"))

    for date in dates:
        day_of_week = datetime.strptime(date,"%d%m%Y").weekday()
        if day_of_week in [5,6]:
            continue

        data = pd.read_csv(f"/Users/vinayak/Desktop/IIT Roorkee/Downloading Bhavcopy/bhav{date}.csv")
        df = pd.DataFrame(data)

        df['DATE1'] = pd.to_datetime(df['DATE1'], format='%d-%b-%Y').dt.strftime('%Y-%m-%d')
        for index, row in df.iterrows():
            cursor.execute("""
                INSERT INTO StockData (SYMBOL, SERIES, DATE1, PREV_CLOSE, OPEN_PRICE, HIGH_PRICE, LOW_PRICE, LAST_PRICE, CLOSE_PRICE, AVG_PRICE, TTL_TRD_QNTY, TURNOVER_LACS, NO_OF_TRADES, DELIV_QTY, DELIV_PER)
                VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
            """, tuple(row))
        df.to_sql('StockData', con=engine, if_exists='append', index=False)

    connection.commit()
    cursor.close()
    connection.close()


In [7]:
def download_bhavcopy():
    current_date=datetime.now()
    date = current_date.strftime('%d%m%Y')
    # print(date)
    dates=[]
    dates.append(current_date.strftime("%d%m%Y"))

    for date in dates:
        day_of_week = datetime.strptime(date,"%d%m%Y").weekday()
        if day_of_week in [5,6]:
            continue

        url=f"https://archives.nseindia.com/products/content/sec_bhavdata_full_{date}.csv"

        response=requests.get(url,timeout=10)
        if(response.status_code==200):
            with open(f"/Users/vinayak/Desktop/IIT Roorkee/Downloading Bhavcopy/bhav{date}.csv",'wb') as file:
                for chunk in response.iter_content():
                    file.write(chunk)

        csv_file_path = f"/Users/vinayak/Desktop/IIT Roorkee/Downloading Bhavcopy/bhav{date}.csv"
        df = pd.read_csv(csv_file_path)

        # Remove leading and trailing whitespaces from column names
        df.columns = df.columns.str.strip()
        df = df.apply(lambda x: x.str.strip() if x.dtype == 'object' else x)

        # Filter rows where 'SERIES' is either 'EQ' or 'BE'
        filtered_df = df[df['SERIES'].isin(['EQ', 'BE'])]
        # Overwrite the original CSV file with the filtered data
        filtered_df.to_csv(csv_file_path, index=False)

        inserting_sql()


In [8]:

schedule.every().day.at('19:45').do(download_bhavcopy)

while True:
    schedule.run_pending()
    time.sleep(10)

KeyboardInterrupt: 